# Exploratory Data Analysis

In [2]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
sns.set()

## Importing data using Pandas

In [3]:
df_deli = pd.read_csv('Data/1.Data Exploration/delivery.csv', index_col='timestamp')
df_cons = pd.read_csv('Data/1.Data Exploration/consumption.csv', index_col='timestamp')
df_info = pd.read_csv('Data/1.Data Exploration/houses_info.csv', index_col='ID-nummer')

display(df_deli.head())
display(df_cons.head())
display(df_info.head())

,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,...,H24,H25,H26,H27,H28,H29,H30,H31,H32,H33
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-07-11 00:15:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 00:30:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 00:45:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 01:00:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 01:15:00,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,H01,H02,H03,H04,H05,H06,H07,H08,H09,H10,...,H24,H25,H26,H27,H28,H29,H30,H31,H32,H33
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-07-11 00:15:00,0.038,0.039,0.044,0.024,NaN,0.039,0.025,0.006,0.294,0.036,...,0.053,0.018,0.076,0.070,0.036,0.014,0.044,0.416,0.023,0.053
2017-07-11 00:30:00,0.018,0.027,0.034,0.024,NaN,0.051,0.017,0.007,0.165,0.025,...,0.049,0.019,0.083,0.061,0.027,0.026,0.034,0.170,0.031,0.029
2017-07-11 00:45:00,0.028,0.014,0.026,0.018,NaN,0.076,0.018,0.006,0.107,0.030,...,0.043,0.018,0.292,0.054,0.018,0.011,0.038,0.075,0.025,0.028
2017-07-11 01:00:00,0.026,0.014,0.474,0.021,NaN,0.080,0.016,0.006,0.105,0.036,...,0.058,0.019,0.049,0.034,0.022,0.026,0.032,0.099,0.024,0.041
2017-07-11 01:15:00,0.018,0.014,0.084,0.117,NaN,0.438,0.023,0.006,0.103,0.015,...,0.045,0.018,0.048,0.208,0.015,0.017,0.043,0.054,0.025,0.032


,concept,PV-aantal,personen
ID-nummer,,,
H01,E,17,4
H02,E,14,2
H03,WP,9,4
H04,WP,11,1
H05,WP,12,4


## Data Cleaning

During the data exploration process, we found out the following problems:
1. There are houses with quite less data than the others.
2. There is missing data and outliers because of the malfunction of the smart meter.

We will handle these problems in this notebook.

### Handling outliers

First of all, we will set the outliers of our dataset to NaN values, which will be deleted later.

In order to find those outliers, we will follow some instructions:
* **Energy Delivery:** These solar panels are able to produce a maximum of 250 watts/hour (0.25 kwh) each one. Since the smart meter sums up all the energy delivery every 15 minutes, we will compute what is the maximum production for each dwelling taking into account the number of solar panels. Then, every value over the maximum will be set as an outlier.

\begin{align}
outlier > 0.25kwh * 0.25h * num\_solar\_panels
\end{align}

* **Energy Consumption:** The maximun consumption that any dwelling can put up with is measured by the following equation, where the kWh are converted to W (0.001) in an interval of 15 minutes (0.25h), where the voltage that the dwelling can handle is 230V and 75A.

\begin{align}
outlier > 0.001kwh * 0.25h * 75A * 230V
\end{align}

In order to check how many outliers will be dropped by the cleaning process, let's save first the number of NaN values on each dwelling.

In [3]:
deli_nan = df_deli.isna().sum()
cons_nan = df_cons.isna().sum()

#### Energy Delivery

In [4]:
for house in range(1,34):
    if house < 10:
        house_no = 'H0'+ str(house)
    else:
        house_no = 'H'+ str(house)
            
    df_deli.loc[df_deli[house_no] > (0.25 * 0.25 * df_info.loc['H01','PV-aantal']), house_no] = np.nan

#### Energy Consumption

In [5]:
for house in range(1,34):
    if house < 10:
        house_no = 'H0'+ str(house)
    else:
        house_no = 'H'+ str(house)
            
    df_cons.loc[df_cons[house_no] > (0.001 * 0.25 * 75 * 230), house_no] = np.nan

The number of outliers on each dwelling are the following:

In [6]:
# Energy delivery
df_deli.isna().sum() - deli_nan

H01    6
H02    6
H03    5
H04    5
H05    4
H06    5
H07    8
H08    1
H09    6
H10    3
H11    5
H12    5
H13    5
H14    5
H15    7
H16    5
H17    4
H18    6
H19    8
H20    4
H21    6
H22    8
H23    5
H24    2
H25    6
H26    4
H27    4
H28    4
H29    9
H30    6
H31    6
H32    6
H33    6
dtype: int64

In [7]:
# Energy consumption
df_cons.isna().sum() - cons_nan

H01    6
H02    5
H03    6
H04    3
H05    6
H06    7
H07    5
H08    3
H09    5
H10    3
H11    4
H12    6
H13    5
H14    5
H15    5
H16    7
H17    6
H18    7
H19    6
H20    3
H21    6
H22    5
H23    5
H24    4
H25    3
H26    5
H27    8
H28    3
H29    5
H30    7
H31    4
H32    5
H33    3
dtype: int64

Once we have set the outliers to NaN values, we will check how many outliers each dwelling has in both datasets.

In [8]:
df_deli.isna().sum() + df_cons.isna().sum()

H01     3780
H02     4325
H03     4211
H04     4304
H05    21268
H06     4156
H07     4335
H08     3262
H09     4133
H10    36666
H11    13143
H12     3965
H13     4040
H14    31625
H15     4434
H16     4260
H17     3794
H18     4299
H19     9538
H20     4197
H21     4134
H22     5295
H23     3946
H24     3426
H25     3589
H26     3591
H27    15718
H28     3617
H29     4526
H30    33307
H31     3794
H32     3795
H33     3971
dtype: int64

#### Dropping dwellings

As we can see above, there are some dweillings which have quite less data than the other ones. As we are going to delete every  record which contains at least one NaN, we will delete these dwellings from the dataset in order to not lose too much data. 

These dweillings will be the following:
* House 5
* House 10
* House 11
* House 14
* House 27
* House 30

In [9]:
df_deli = df_deli.drop(columns=['H05', 'H10', 'H11', 'H14', 'H27', 'H30'])
df_cons = df_cons.drop(columns=['H05', 'H10', 'H11', 'H14', 'H27', 'H30'])
df_info = df_info.drop(['H05', 'H10', 'H11', 'H14', 'H27', 'H30'])

In [10]:
display(df_deli.head())
display(df_cons.head())
display(df_info.head())

,H01,H02,H03,H04,H06,H07,H08,H09,H12,H13,...,H22,H23,H24,H25,H26,H28,H29,H31,H32,H33
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-07-11 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-07-11 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,H01,H02,H03,H04,H06,H07,H08,H09,H12,H13,...,H22,H23,H24,H25,H26,H28,H29,H31,H32,H33
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-07-11 00:15:00,0.038,0.039,0.044,0.024,0.039,0.025,0.006,0.294,0.137,0.058,...,0.024,0.115,0.053,0.018,0.076,0.036,0.014,0.416,0.023,0.053
2017-07-11 00:30:00,0.018,0.027,0.034,0.024,0.051,0.017,0.007,0.165,0.402,0.060,...,0.013,0.101,0.049,0.019,0.083,0.027,0.026,0.170,0.031,0.029
2017-07-11 00:45:00,0.028,0.014,0.026,0.018,0.076,0.018,0.006,0.107,0.019,0.363,...,0.025,0.108,0.043,0.018,0.292,0.018,0.011,0.075,0.025,0.028
2017-07-11 01:00:00,0.026,0.014,0.474,0.021,0.080,0.016,0.006,0.105,0.024,0.049,...,0.024,0.090,0.058,0.019,0.049,0.022,0.026,0.099,0.024,0.041
2017-07-11 01:15:00,0.018,0.014,0.084,0.117,0.438,0.023,0.006,0.103,0.013,0.198,...,0.014,0.246,0.045,0.018,0.048,0.015,0.017,0.054,0.025,0.032


,concept,PV-aantal,personen
ID-nummer,,,
H01,E,17,4
H02,E,14,2
H03,WP,9,4
H04,WP,11,1
H06,E,14,4


#### Deleting NaN values

In order to be able to compare the dwellings, we will delete those records (rows) where there is at least one NaN value.

In [11]:
display(df_deli.shape)
display(df_cons.shape)

(66240, 27)

(66240, 27)

In [12]:
df_deli.columns = df_deli.columns.map(lambda x: str(x) + '_d')
df_cons.columns = df_cons.columns.map(lambda x: str(x) + '_c')

df_full = df_deli.join(df_cons, how='outer')
df_full.dropna(inplace=True)

df_deli = df_full.loc[:, :'H33_d']
df_cons = df_full.loc[:, 'H01_c':]

df_deli.columns = df_deli.columns.map(lambda x: x[:3])
df_cons.columns = df_cons.columns.map(lambda x: x[:3])

In [13]:
display(df_deli.shape)
display(df_cons.shape)

(59510, 27)

(59510, 27)

In [14]:
df_deli.describe()

,H01,H02,H03,H04,H06,H07,H08,H09,H12,H13,...,H22,H23,H24,H25,H26,H28,H29,H31,H32,H33
count,59510.00000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,...,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000
mean,0.10194,0.069091,0.047598,0.061644,0.063065,0.086891,0.038144,0.090842,0.065939,0.057347,...,0.072508,0.054746,0.051912,0.072868,0.066988,0.082966,0.078539,0.085399,0.069896,0.054576
std,0.19545,0.148544,0.099400,0.123540,0.130873,0.163505,0.084816,0.167860,0.141269,0.121562,...,0.146442,0.113309,0.110022,0.141632,0.132638,0.156080,0.149056,0.168368,0.135378,0.109196
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.10500,0.045000,0.033000,0.053000,0.051000,0.093000,0.023000,0.105000,0.053000,0.045000,...,0.060000,0.043000,0.033000,0.074000,0.061000,0.087000,0.083000,0.078000,0.071000,0.048000
max,0.91100,0.770000,0.503000,0.640000,0.768000,0.787000,0.467000,0.994000,0.814000,0.826000,...,0.719000,0.623000,0.576000,0.736000,0.674000,0.749000,0.946000,0.843000,0.694000,0.554000


In [15]:
df_cons.describe()

,H01,H02,H03,H04,H06,H07,H08,H09,H12,H13,...,H22,H23,H24,H25,H26,H28,H29,H31,H32,H33
count,59510.000000,59510.000000,59510.00000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,...,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000,59510.000000
mean,0.121020,0.132009,0.07911,0.062257,0.092334,0.112259,0.131364,0.128476,0.108573,0.104593,...,0.062667,0.064412,0.104995,0.068297,0.092079,0.119657,0.115883,0.089110,0.115918,0.062165
std,0.234742,0.258810,0.12335,0.096648,0.152262,0.265581,0.244691,0.261763,0.184223,0.166663,...,0.204278,0.087908,0.134051,0.253692,0.126514,0.379426,0.261039,0.122551,0.328675,0.104296
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.014000,0.001000,0.00100,0.001000,0.006000,0.003000,0.008000,0.000000,0.003000,0.003000,...,0.000000,0.000000,0.014000,0.000000,0.008000,0.000000,0.000000,0.000000,0.000000,0.004000
50%,0.032000,0.028000,0.03100,0.031000,0.043000,0.018000,0.026000,0.024000,0.029000,0.030000,...,0.023000,0.033000,0.059000,0.015000,0.041000,0.014000,0.015000,0.042000,0.025000,0.030000
75%,0.090000,0.079000,0.08600,0.064000,0.087000,0.062000,0.126000,0.093000,0.140000,0.155000,...,0.042000,0.084000,0.133000,0.025000,0.124000,0.023000,0.036000,0.103000,0.047000,0.047000
max,2.084000,1.905000,1.38500,0.979000,1.914000,2.606000,2.192000,2.137000,2.067000,1.720000,...,2.049000,0.947000,1.812000,2.512000,1.447000,2.509000,2.240000,1.163000,2.501000,1.093000


### Exporting the datasets

Now that we have cleaned the datasets, we will export them to be used in the following steps.

In [16]:
df_deli.to_csv('Data/2.Data Cleaning/delivery.csv')
df_cons.to_csv('Data/2.Data Cleaning/consumption.csv')
df_info.to_csv('Data/2.Data Cleaning/houses_info.csv')